In [2]:
import numpy as np
from IPython.display import display, Markdown

State space $S = \{\text{hungry}, \text{full}\}$.

Action space $A = \{\text{ignore}, \text{feed}\}$.

Initial state distribution $p_{S_0} = \begin{bmatrix}1/2 & 1/2\end{bmatrix}^T$.

Rewards = $\{-3, -2, 1, 2\}$

In [3]:
p_S0 = np.array([1/2, 1/2])

## Dynamics
$$p(s',r\mid s, a) = \mathrm{Pr}[S_{t+1} = s', R_{t+1}=r\mid S_t = s, A_t = a]$$


In [4]:
# Dynamics as a matrix
dynamics_matrix = [[0, 1/3,   0,   0], # (hungry, -3)
                   [1,   0, 3/4,   0], # (hungry, -2)
                   [0, 2/3,   0,   1], # (full, 1)
                   [0,   0, 1/4,   0], # (full, 2)
                   ]
dynamics_matrix = np.array(dynamics_matrix) 

## Transition Probability Matrix
$$
\begin{align*}
p(s'\mid s,a) &= \mathrm{Pr}\left[S_{t+1} = s' \mid S_t = s, A_t = a\right]\\
&= \sum_{r \in R} p(s', r \mid s, a)
\end{align*}
$$

In [5]:
def get_p_sp_given_s_a(dynamics_matrix):
    p_sp_given_s_a = np.array([[1,1,0,0], [0,0,1,1]]) @ dynamics_matrix
    return p_sp_given_s_a

In [6]:
p_sp_given_s_a = get_p_sp_given_s_a(dynamics_matrix)
p_sp_given_s_a

array([[1.        , 0.33333333, 0.75      , 0.        ],
       [0.        , 0.66666667, 0.25      , 1.        ]])

## Expected Reward given State-Action Pair
$$
\begin{align*}
r(s,a) &= \mathbb{E}[R_{t + 1} \mid S_t = s, A_t = a]\\
&= \sum_{s' \in S, r \in R} r \cdot p(s', r \mid s, a)
\end{align*}
$$

In [7]:
def get_r(dynamics_matrix):
    r = np.sum(dynamics_matrix * np.array([[-3], [-2], [1], [2]]), axis=0) # Keep in vector form just in case.
    return r

In [8]:
r = get_r(dynamics_matrix)
r

array([-2.        , -0.33333333, -1.        ,  1.        ])

## Expected Reward given State, Action and Next State.
$$
\begin{align*}
r(s,a,s') &= \mathbb{E}[R_{t + 1} \mid S_t = s, A_t = a, S_{t + 1} = s']\\
&= \frac{\sum_{r \in R}r \cdot p(s', r \mid s, a)}{p(s'\mid s, a)}
\end{align*}
$$

In [9]:
def get_r_sp(dynamics_matrix, p_sp_given_s_a):
    r_sp = np.array([[1,1,0,0],[0,0,1,1]]) @ (dynamics_matrix * np.array([[-3], [-2], [1], [2]]))
    r_sp = np.nan_to_num(r_sp / p_sp_given_s_a)
    return r_sp

In [10]:
r_sp = get_r_sp(dynamics_matrix, p_sp_given_s_a)
r_sp

C:\Users\_Kookie\AppData\Local\Temp\ipykernel_10788\3337454226.py:3: RuntimeWarning: invalid value encountered in divide
  r_sp = np.nan_to_num(r_sp / p_sp_given_s_a)


array([[-2., -3., -2.,  0.],
       [ 0.,  1.,  2.,  1.]])

## Defining Policy $\pi$

In [11]:
pi = np.array([1/4, 3/4, 5/6, 1/6])
pi

array([0.25      , 0.75      , 0.83333333, 0.16666667])

## Initial State-Action Distribution, According to $\pi$
$$
\begin{align*}
p_{S_0, A_0, \pi}(s, a) &= \mathrm{Pr}_\pi[S_0 = s, A_0 = a]\\
&= p_{S_0}(s)\pi(a\mid s)
\end{align*}
$$

In [12]:
def get_p_S0_A0_pi(pi, p_S0):
    pi_matrix = pi.reshape(2,2)
    p_S0_A0_pi = p_S0.reshape(-1,1) * pi_matrix
    return p_S0_A0_pi

In [13]:
p_S0_A0_pi = get_p_S0_A0_pi(pi, p_S0)
p_S0_A0_pi

array([[0.125     , 0.375     ],
       [0.41666667, 0.08333333]])

## Transition Probability between States, According to $\pi$
$$
\begin{align*}
p_\pi(s'\mid s) &= \mathrm{Pr}_\pi[S_{t + 1} = s'\mid S_t = s]\\
&= \sum_{a \in A}p(s'\mid s, a)\pi(a\mid s)
\end{align*}
$$

In [14]:
def get_p_pi_sp_given_s(pi, p_sp_given_s_a):
    p_pi_sp_given_s = (p_sp_given_s_a * pi) @ np.array([[1,1,0,0], [0,0,1,1]]).T
    return p_pi_sp_given_s

In [15]:
p_pi_sp_given_s = get_p_pi_sp_given_s(pi, p_sp_given_s_a)
p_pi_sp_given_s

array([[0.5  , 0.625],
       [0.5  , 0.375]])

## Transition Probability between State-Action Pairs, According to $\pi$
$$
\begin{align*}
p_\pi(s', a'\mid s, a) &= \mathrm{Pr}_\pi[S_{t + 1} = s', A_{t + 1} = a' \mid S_t = s, A_t = a]\\
&= \pi(a' \mid s')p(s'\mid s, a)
\end{align*}
$$

In [16]:
def get_p_pi_sp_given_s_a(pi, p_sp_given_s_a):
    p_pi_sp_ap_given_s_a = pi.reshape(-1,1) * np.repeat(p_sp_given_s_a,2, axis=0)
    return p_pi_sp_ap_given_s_a

In [17]:
p_pi_sp_ap_given_s_a = get_p_pi_sp_given_s_a(pi, p_sp_given_s_a)
p_pi_sp_ap_given_s_a

array([[0.25      , 0.08333333, 0.1875    , 0.        ],
       [0.75      , 0.25      , 0.5625    , 0.        ],
       [0.        , 0.55555556, 0.20833333, 0.83333333],
       [0.        , 0.11111111, 0.04166667, 0.16666667]])

## Expected State Reward, According to $\pi$
$$
\begin{align*}
r_\pi(s) &= \mathbb{E}_\pi[R_{t+1}\mid S_t = s]\\
&= \sum_{a \in A}r(s,a)\pi(a\mid s)
\end{align*}
$$

In [18]:
def get_r_pi(pi, r):
    pi_matrix = pi.reshape(2,2)
    r_pi = np.diag(r.reshape(2,2) @ pi_matrix.T)
    return r_pi

In [19]:
r_pi = get_r_pi(pi, r)
r_pi

array([-0.75      , -0.66666667])

## Discounted Return
Episodic task without discount.
$$
G_t = \sum_{\tau = t+1}^TR_{\tau}
$$
Episodic task with discount $\gamma \in [0,1]$:
$$
G_t = \sum_{\tau = 0}^\infty \gamma^\tau R_{t+\tau+1}
$$
Recursive Relationship:
$$
G_t = R_{t+1} + \gamma G_{t + 1}
$$

## Value
State Value - expected return starting from $s$, then following policy $\pi$.
$$
v_\pi(s) = \mathbb{E}_\pi[G_t\mid S_t = s]
$$
Action Value - expected return starting from $(s,a)$, then following $\pi$.
$$
q_\pi(s,a) = \mathbb{E}_\pi[G_t\mid S_t = s, A_t = a]
$$
### Properties of Value
Using action-value pairs to back up state values
$$
v_\pi(s) = \sum_{a \in A}\pi(a\mid s)q_\pi(s,a)
$$
In other words: $v_\pi(S_t) = \mathbb{E}_\pi[q_\pi(S_t,A_t)]$.

Vector form:
$$
\vec{v}_\pi = \vec{\pi} \odot \vec{q}_\pi
$$
Using state values to back up action values:
$$
\begin{align*}
q_\pi(s,a) &= r(s,a) + \gamma\sum_{s' \in S}p(s'\mid s,a)v_\pi(s')\\
&= \sum_{s' \in S^+, r \in R}p(s',r\mid s,a)[r + \gamma v_\pi(s')]
\end{align*}
$$
In other words, $q_\pi(S_t,A_t) = \mathbb{E}[R_{t+1} + \gamma v_\pi(S_{t+1})]$.

Vector form:
$$
\vec{q}_\pi = \vec{r} + \gamma P^\top_{S_{t+1}\mid S_t,A_t}\vec{v}_\pi
$$
- $\vec{v}_\pi = (v_\pi(s): s \in S)^\top \in \mathbb{R}^{|S|}$
- $\vec{q}_\pi = (q_\pi(s,a):(s,a) \in S \times A)^\top \in \mathbb{R}^{|S||A|}$
- $\vec{\pi} = (\pi(a\mid s): (s,a) \in S \times A)^\top \in \mathbb{R}^{|S||A|}$
- $\vec{r} = (r(s,a) : (s,a) \in S \times A)^\top \in \mathbb{R}^{|S||A|}$
- $P_{S_{t+1}\mid S_t, A_t} = (p(s'\mid s,a): s' \in S, (s,a) \in S \times A) \in [0,1]^{|S| \times |S||A|}$

In [20]:
p_sp_given_s_a.shape, r.shape

((2, 4), (4,))

## Bellman Expectation Equations
The two relations above combine to form the two relations below.

### Use state values at time $t+1$ to back up the state values at time $t$
$$
v_\pi(s) = r_\pi(s) + \gamma \sum_{s' \in S}p_\pi(s'\mid s)v_\pi(s')
$$
In other words: $v_\pi(S_t) = \mathbb{E}_\pi[R_{t+1} + \gamma v_\pi(S_{t+1})]$.

Vector form:
$$
\vec{v}_\pi = \vec{r}_\pi + \gamma P^\top_{S_{t+1}\mid S_t;\pi}\vec{v}_\pi
$$
where $\vec{r}_\pi = (r_\pi(s): s\in S)^\top \in \mathbb{R}^{|S|}$.

### Use action values at time $t+1$ to represent the action values at time $t$
$$
\begin{align*}
q_\pi(s,a) &= r(s,a) + \gamma \sum_{s'\in S, a'\in A}p_\pi(s', a'\mid s, a)q_\pi(s',a')\\
&= \sum_{s'\in S, r\in R}p(s', r\mid s, a)\left[r + \gamma\sum_{a'\in A}\pi(a'\mid s')q_\pi(s', a')\right]
\end{align*}
$$
In other words, $q_\pi(S_t, A_t) = \mathbb{E}_\pi[R_{t+1} + \gamma q_\pi(S_{t+1}, A_{t+1})]$

Vector form:
$$
\vec{q}_\pi = \vec{r} + \gamma P^\top_{S_{t+1}, A_{t+1}\mid S_t, A_t;\pi} \vec{q}_\pi
$$

Rearranged relations:
$$
\begin{align*}
\vec{v}_\pi &= (I - \gamma P^\top_{S_{t+1}\mid S_t;\pi})^{-1}\vec{r}_\pi\\
\vec{q}_\pi &= (I - \gamma P^\top_{S_{t+1}, A_{t+1}\mid S_t, A_t;\pi})^{-1}\vec{r}
\end{align*}
$$

### Approach 1
1. Use $\vec{v}_\pi = (I - \gamma P^\top_{S_{t+1}\mid S_t;\pi})^{-1}\vec{r}_\pi$ to obtain $\vec{v}_\pi$.
2. Use $\vec{q}_\pi = \vec{r} + \gamma P^\top_{S_{t+1}\mid S_t, A_t; \pi}\vec{v}_\pi$ to obtain $\vec{q}_\pi$

In [21]:
# Feed and Full Example
def get_vq1(p_pi_sp_given_s, p_sp_given_s_a, gamma, r_pi, r):
    v_pi = np.linalg.inv(np.eye(2) - gamma * p_pi_sp_given_s.T) @ r_pi
    q_pi = r + (gamma * p_sp_given_s_a.T) @ v_pi
    return v_pi, q_pi

In [22]:
gamma = 4/5
v_pi1, q_pi1 = get_vq1(p_pi_sp_given_s, p_sp_given_s_a, gamma, r_pi, r)
v_pi1, q_pi1

(array([-3.59848485, -3.52272727]),
 array([-4.87878788, -3.17171717, -3.86363636, -1.81818182]))

### Approach 2
1. Use $\vec{q}_\pi = (I - \gamma P^\top_{S_{t+1}\mid S_t, A_t;\pi})^{-1}\vec{r}$ to obtain $\vec{q}_\pi$.
2. Use $\vec{v}_\pi = \vec{\pi} \odot \vec{q}_\pi$ to obtain $\vec{v}_\pi$.

In [23]:
def get_vq2(p_pi_sp_given_s, p_sp_given_s_a, gamma, pi, r):
    q_pi = np.linalg.inv(np.eye(4) - gamma * p_pi_sp_ap_given_s_a.T) @ r
    v_pi = np.array([[1,1,0,0],[0,0,1,1]]) @ (pi * q_pi)
    return v_pi, q_pi

In [24]:
v_pi2, q_pi2 = get_vq2(p_pi_sp_given_s, p_sp_given_s_a, gamma, pi, r)
v_pi2, q_pi2

(array([-3.59848485, -3.52272727]),
 array([-4.87878788, -3.17171717, -3.86363636, -1.81818182]))

Here we see that both approaches achieve the same results.

## Initial Expected Returns using Values
Expected return at $t = 0$:
$$
\begin{align*}
g_\pi &= \mathbb{E}_{S_0 \sim p_{S_0}}[v_\pi(S_0)]\\
&= \vec{p}_{S_0}^\top \vec{v}_\pi
\end{align*}
$$

In [25]:
def get_g_pi(p_S0, v_pi):
    g_pi = p_S0 @ v_pi.reshape(-1,1)
    return g_pi[0]

In [26]:
g_pi = get_g_pi(p_S0, v_pi1)
g_pi

np.float64(-3.5606060606060614)

## Policy Improvement Theorem
If for all $s \in S$,
$$
v_\pi(s) \leq \sum_a \pi'(a\mid s)q_\pi(s, a) = \mathbb{E}_{A\sim \pi'(s)}[q_\pi(s,A)]
$$
then $\pi \preccurlyeq \pi'$. I.e: $v_\pi(s) \leq v_{\pi'}(s)$.

Additionally, if there is a state $s \in S$ such that the former inequality holds, there is a state $s \in S$ such that the latter inequality also holds.

In [27]:
def policy_le(v_p1, q_p1, p2):
    return np.all(v_p1 <= (np.array([[1,1,0,0], [0,0,1,1]]) @ (p2 * q_p1)))

In [28]:
pi2 = np.array([0,1,0,1])

In [29]:
policy_le(v_pi1, q_pi1, pi2)

np.True_

### Check if a Policy is Optimal
- For each state-action pair $s \in S$ and $a \in A(s)$: if $\pi(a\mid s) > 0$ and $q_\pi(s, a) < \max_{a' \in A}q_\pi(s, a')$, then the policy is not optimal.
- Otherwise, policy $\pi$ is optimal and cannot be further improved.

In [30]:
def is_policy_optimal(pi, q_pi):
    m = int(len(q_pi) ** (1/2))
    mask = (pi.reshape(m,m) > 0) & (q_pi.reshape(m,m) < np.max(q_pi.reshape(m,m), axis=1).reshape(-1,1))
    return not np.any(mask)

In [31]:
is_policy_optimal(pi, q_pi1)

False

In [32]:
p_pi2_sp_given_s = get_p_pi_sp_given_s(pi2,p_sp_given_s_a)
r_pi2 = get_r_pi(pi2, r) 
v_pi_det, q_pi_det = get_vq1(p_pi2_sp_given_s, p_sp_given_s_a, gamma, r_pi2, r)
v_pi_det, q_pi_det

(array([3.18181818, 5.        ]),
 array([0.54545455, 3.18181818, 1.90909091, 5.        ]))

In [33]:
is_policy_optimal(pi2, q_pi_det)

True

### Policy Improvement Algorithm
- For each state $s \in S$, set $\pi'(s) \leftarrow \arg\max_{a \in A}q_\pi(s,a)$.

In [34]:
def improve_policy(pi, q_pi):
    pi_matrix = pi.reshape(2,2)
    q_pi_matrix = q_pi.reshape(2,2)
    new_pi_matrix = np.zeros(pi_matrix.shape)
    indices = np.argmax(q_pi_matrix, axis=1)
    new_pi_matrix[:, indices] = 1
    new_pi = new_pi_matrix.reshape(-1)
    return new_pi

In [35]:
pi_improved = improve_policy(pi, q_pi1)

In [36]:
pi_improved

array([0., 1., 0., 1.])

## Visitation Frequency
Determine how many times a state or state-action pair will be visited, with weights - **discounted visitation frequency** a.k.a. **discounted distribution**.

Recall $T$ is a random variable denoting the stop time.

### Discounted State Visitation Frequency (Discounted State Distribution)
Episodic Task:
$$
\eta_\pi(s) = \sum_{t = 0}^\infty \mathrm{Pr}_\pi[T = t]\sum_{\tau = 0}^{t-1}\gamma^\tau \mathrm{Pr}_\pi[S_\tau = s]
$$
Sequential Task:
$$
\eta_\pi(s) = \sum_{\tau = 0}^\infty \gamma^\tau \mathrm{Pr}_\pi[S_\tau = s]
$$

### Discounted State-Action Visitation Frequency (Discounted State-Action Distribution)
Episodic Task:
$$
\rho_\pi(s,a) = \sum_{t = 0}^\infty\mathrm{Pr}_\pi[T = t]\sum_{\tau = 0}^{t - 1}\gamma^\tau \mathrm{Pr}_\pi[S_\tau = s, A_\tau = a]
$$
Sequential Task:
$$
\rho_\pi(s,a) = \sum_{\tau = 0}^\infty \gamma^\tau \mathrm{Pr}_\pi[S_\tau = s, A_\tau = a]
$$

### Properties
Episodic Task:
$$
\sum_{s \in S}\eta_\pi(s) = \sum_{s \in S, a \in A(s)}\rho_\pi(s, a) = \mathbb{E}_\pi\left[\frac{1 - \gamma^T}{1 - \gamma}\right]
$$
Discounted Visitation/Discounted Distribution may not be a probability distribution.

Sequential Task:
$$
\sum_{s \in S}\eta_\pi(s) = \sum_{s \in S, a \in A(s)}\rho_\pi(s,a) = \frac{1}{1 - \gamma}
$$

#### Feed and Full Example
The environment is a sequential task.
$$
\begin{align*}
\eta_\pi(\mathrm{hungry}) &= \sum_{\tau = 0}^{\infty}\gamma^\tau\mathrm{Pr}_\pi[S_\tau = \mathrm{hungry}]
\end{align*}
$$

### Manual Calculation of $\eta_\pi$ - Directly from Definition
We can write
```math

```

In [37]:
def get_p_pi_s(p_s0, tau):
    return (np.linalg.matrix_power(p_pi_sp_given_s, tau) @ p_S0.reshape(-1,1)).reshape(-1)

In [38]:
eta_pi = np.sum(np.array([gamma ** tau * get_p_pi_s(p_S0, tau) for tau in range(10000)]), axis=0)
eta_pi

array([2.72727273, 2.27272727])

### Manual Calculation of $\rho_\pi$ - Directly from Definition

In [39]:
rho_pi = np.sum(np.array([gamma ** tau * pi.reshape(2,2) * np.repeat(get_p_pi_s(p_S0, tau).reshape(-1,1), 2, axis=1) for tau in range(10000)]), axis=0).reshape(-1)
rho_pi

array([0.68181818, 2.04545455, 1.89393939, 0.37878788])

## Other Properties
Using discounted state-action visitation frequency to back up discounted state visitation frequency.
$$
\eta_\pi(s) = \sum_{a \in A(s)}\rho_\pi(s,a)
$$
Vectorised:
$$
\vec{\eta}_\pi = \mathbf{1}_{S \mid S, A}\vec{\rho}_\pi
$$
Where:
- $\mathbf{1}_{S \mid S, A} = (1: s \in S, (s,a) \in S \times A) \in \{1\}^{|S| \times |S||A|}$.

In [40]:
eta_pi

array([2.72727273, 2.27272727])

In [41]:
np.sum(rho_pi.reshape(2,2), axis=1)

array([2.72727273, 2.27272727])

Use discounted state visitation frequency and poliicy to back up discounted state-action visitation frequency.
$$
\rho_\pi(s,a) = \eta_\pi(s)\pi(a\mid s)
$$

In [42]:
rho_pi

array([0.68181818, 2.04545455, 1.89393939, 0.37878788])

In [43]:
(np.repeat(eta_pi.reshape(-1,1), 2, axis=1) * pi.reshape(2,2)).reshape(-1)

array([0.68181818, 2.04545455, 1.89393939, 0.37878788])

Using discounted state-action visitation frequency and dynamics to back up discounted state visitation frequency.
$$
\eta_\pi(s') = p_{S_0}(s') + \gamma \sum_{s \in S, a \in A(s)}p(s'\mid s, a) \rho_\pi(s, a)
$$
Vectorised:
$$
\vec{\eta}_\pi = \vec{p}_{S_0} + \gamma P_{S_{t+1}\mid S_t, A_t}\vec{\rho}_\pi
$$

In [44]:
eta_pi

array([2.72727273, 2.27272727])

In [45]:
(p_S0 + gamma * p_sp_given_s_a @ rho_pi.reshape(-1,1))[:,0]

array([2.72727273, 2.27272727])

## Bellman Expectation Equations
Use discounted state distribution at time $t$ to back up discounted state distribution at time $t+1$:
$$
\eta_\pi(s') = p_{S_0}(s') + \gamma\sum_{s \in S}p_\pi(s'\mid s)\eta_\pi(s)
$$
Vectorised:
$$
\vec{\eta}_\pi = \vec{p}_{S_0} + \gamma P_{S_{t+1}\mid S_t; \pi}\vec{\eta}_\pi
$$

In [46]:
eta_pi

array([2.72727273, 2.27272727])

In [47]:
p_S0 + gamma * p_pi_sp_given_s @ eta_pi

array([2.72727273, 2.27272727])

Use discounted state-action distribution at time $t$ to back up the discounted state-action distribution at time $t+1$.
$$
\rho_\pi(s', a') = p_{S_0, A_0; \pi}(s', a') + \gamma\sum_{s\in S, a \in A(s)}p_\pi(s', a'\mid s, a)\rho_\pi(s,a)
$$
Vectorised:
$$
\vec{\rho}_\pi = \vec{p}_{S_0, A_0;\pi} + \gamma P_{S_{t+1}, A_{t+1}\mid S_t, A_t;\pi}\vec{\rho}_\pi
$$

In [48]:
rho_pi

array([0.68181818, 2.04545455, 1.89393939, 0.37878788])

In [49]:
np.diag(p_S0_A0_pi.reshape(-1) + gamma * p_pi_sp_ap_given_s_a @ rho_pi.reshape(-1,1))

array([0.68181818, 2.04545455, 1.89393939, 0.37878788])

## Calculating Visitation Frequency
The following methods allow the computation of $\vec{\rho}_\pi$ and $\vec{eta}_\pi$ using purely linear algebra - no loops, and more pure exactness.
### Approach 1
1. Rearrange $\vec{\eta}_\pi$ Bellman Expectation Equation into $\vec{\eta}_\pi = (I - \gamma P_{S_{t+1}\mid S_t})^{-1}\vec{p}_{S_0}$ to obtain $\vec{\eta}_\pi$.
2. Use $\rho_\pi(s,a) = \eta_\pi(s)\pi(a\mid s)$ to obtain $\vec{\rho}_\pi$.

In [50]:
def get_eta_rho1(gamma, p_pi_sp_given_s, p_S0, pi):
    eta_pi = np.linalg.inv(np.eye(2) - gamma * p_pi_sp_given_s) @ p_S0
    rho_pi = eta_pi.reshape(-1,1) * pi.reshape(2,2)
    return eta_pi, rho_pi.reshape(-1)

In [51]:
get_eta_rho1(gamma, p_pi_sp_given_s, p_S0, pi)

(array([2.72727273, 2.27272727]),
 array([0.68181818, 2.04545455, 1.89393939, 0.37878788]))

### Approach 2
1. Rearrange $\vec{\rho}_\pi$ Bellman Expectation Equation into $\vec{\rho}_\pi = (I - \gamma P_{S_{t + 1}, A_{t + 1}\mid S_t, A_t; \pi})^{-1}\vec{p}_{S_0, A_0; \pi}$.
2. Use $\vec{\eta}_\pi = I_{S_{t+1}\mid S_{t}, A_t}\vec{\rho}_\pi$ to obtain $\vec{\eta}_\pi$.

In [52]:
def get_eta_rho2(gamma, p_pi_sp_ap_given_s_a, p_S0_A0_pi):
    rho_pi = np.linalg.inv(np.eye(4) - gamma * p_pi_sp_ap_given_s_a) @ p_S0_A0_pi.reshape(-1,1)
    eta_pi = np.sum(rho_pi.reshape(2,2), axis=1)
    return eta_pi, rho_pi.reshape(-1)

In [53]:
get_eta_rho2(gamma, p_pi_sp_ap_given_s_a, p_S0_A0_pi)

(array([2.72727273, 2.27272727]),
 array([0.68181818, 2.04545455, 1.89393939, 0.37878788]))

## Equivalence between Visitation Frequency and Policy
We know that discounted visitation frequencies of any $\pi$ satisfy all the following:
$$
\begin{align*}
\eta_\pi(s') &= p_{S_0}(s') + \gamma\sum_{s \in S, a \in A(s)}p(s'\mid s, a)\rho_\pi(s, a)\\
\eta_\pi(s) &= \sum_{a \in A(s)}\rho_\pi(s,a)\\
\rho_\pi(s,a) &\geq 0
\end{align*}
$$
Notice that these equations don't directly use $\pi$. Furthermore, the solution of the above system is bijective with the policy. Therefore, if we find a solution $\vec{\eta}$ and $\vec{\rho}$ that satisfies the equation set, then we can define a policy as follows:
$$
\pi(a\mid s) = \frac{\rho(s, a)}{\eta(s)}
$$
and the policy satisfies:
1. $\eta_\pi(s) = \eta(s)$, and
2. $\rho_\pi(s, a) = \rho(s,a)$.

In [54]:
pi

array([0.25      , 0.75      , 0.83333333, 0.16666667])

In [55]:
eta, rho = get_eta_rho1(gamma, p_pi_sp_given_s, p_S0, pi)
(rho.reshape(2,2) / eta.reshape(-1,1)).reshape(-1)

array([0.25      , 0.75      , 0.83333333, 0.16666667])

## Expectation over visitation Frequency
Given deterministic function $f$, can determine expectation over discounted distribution:
$$
\begin{align*}
    \mathbb{E}_{S\sim \eta_\pi}[f(S)] &= \sum_{s \in S}\eta_\pi(s)f(s)\\
    \mathbb{E}_{(S, A)\sim \rho_\pi}[f(S,A)] &= \sum_{s\in S, a\in A(s)}\rho_\pi(s,a)f(s,a)
\end{align*}
$$
### Example
Expected discounted return at $t = 0$ can be written as
$$
g_\pi = \mathbb{E}_{(S,A)\sim \rho_\pi}[r(S,A)]
$$
Due to this property, we can write
$$
g_\pi = \vec{r}^\top\vec{\rho}_\pi
$$

In [56]:
g_pi

np.float64(-3.5606060606060614)

In [57]:
(r @ rho_pi.reshape(-1,1))[0]

np.float64(-3.5606060606060597)

## Properties of Optimal Values
Use optimal action values at time $t$ to back up optimal state values at time $t$.
$$v_*(s) = \max_{a \in A}q_*(s,a)$$

Use optimal state values at time $t+1$ to back up optimal action values at time $t$.
$$
\begin{align*}
q_*(s,a) &= r(s,a) + \gamma\sum_{s'\in S}p(s'\mid s, a)v_*(s')\\
&= \sum_{s' \in S^+, r\in R}p(s', r\mid s, a)[r + \gamma v_*(s')]
\end{align*}$$
In other words
$$q_*(S_t, A_t) = \mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1})]$$

## Bellman Optimal Equations (BOEs)
Using optimal state values at time $t+1$ to back up optimal state values at time $t$.
$$v_*(s) = \max_{a \in A}\left[r(s,a) + \gamma\sum_{s'\in S}p(s'\mid s, a)v_*(s')\right]$$
In other words:
$$v_*(S_t) = \max_{a \in A}\mathbb{E}[R_{t+1} + \gamma v_*(S_{t+1})]$$

Using optimal action values at time $t+1$ to back up optimal action values at time $t$:
$$q_*(s,a) = r(s,a) + \gamma \sum_{s'\in S}p(s'\mid s, a) \max_{a'\in A} q_*(s', a')$$
In other words:
$$q_*(S_t, A_t) = \mathbb{E}\left[R_{t+1} + \gamma \max_{a'\in A} q_*(S_{t+1}, a')\right]$$

### Feed and Full Example
$$
\begin{align*}
v_*(\text{hungry}) &= \max\{q_*(\text{hungry}, \text{ignore}), q_*(\text{hungry}, \text{feed})\}\\
v_*(\text{full}) &= \max\{q_*(\text{full}, \text{ignore}), q_*(\text{full}, \text{feed})\}\\
q_*(\text{hungry}, \text{ignore}) &= -2 + \frac{4}{5}(1v_*(\text{hungry}) + 0v_*(\text{full}))\\
q_*(\text{hungry}, \text{feed}) &= -\frac{1}{3} + \frac{4}{5}\left(\frac{1}{3}v_*(\text{hungry}) + \frac{2}{3}v_*(\text{full})\right)\\
q_*(\text{full}, \text{ignore}) &= -1 + \frac{4}{5}\left(\frac{3}{4}v_*(\text{hungry}) + \frac{1}{4}v_*(\text{full})\right)\\
q_*(\text{full}, \text{feed}) &= 1 + \frac{4}{5}(0v_*(\text{hungry}) + 1v_*(\text{full}))
\end{align*}
$$
In the code below we will consider case I from the book, where $q_*(\text{hungry},\text{ignore}) > q_*(\text{hungry},\text{feed})$ and $q_*(\text{full}, \text{ignore}) > q_*(\text{full}, \text{feed})$.

In [58]:
r


array([-2.        , -0.33333333, -1.        ,  1.        ])

In [59]:
p_sp_given_s_a

array([[1.        , 0.33333333, 0.75      , 0.        ],
       [0.        , 0.66666667, 0.25      , 1.        ]])

In [93]:
def get_g_star_q_star(idx1, idx2, p_sp_given_s_a, r, gamma):
    assert idx1 in (0,1)
    assert idx2 in (2,3)

    v_star = (np.linalg.inv(np.eye(2) - gamma * p_sp_given_s_a[:, [idx1,idx2]].T) @ r[[idx1,idx2]].reshape(-1,1)).reshape(-1)
    q_star = np.zeros(4)
    q_star[[idx1,idx2]] = v_star

    idx3 = 1 - idx1
    idx4 = 5 - idx2
    q_star[[idx3, idx4]] = (r[[idx3,idx4]].reshape(-1,1) + gamma * p_sp_given_s_a[:, [idx3,idx4]].T @ v_star.reshape(-1,1)).reshape(-1)

    valid = q_star[idx1] > q_star[idx3] and q_star[idx2] > q_star[idx4]

    return v_star, q_star, valid

In [94]:
get_g_star_q_star(0, 2, p_sp_given_s_a, r, gamma)

(array([-10.  ,  -8.75]),
 array([-10.        ,  -7.66666667,  -8.75      ,  -6.        ]),
 np.False_)

In [95]:
get_g_star_q_star(1, 2, p_sp_given_s_a, r, gamma)

(array([-3. , -3.5]), array([-4.4, -3. , -3.5, -1.8]), np.False_)

In [96]:
get_g_star_q_star(0, 3, p_sp_given_s_a, r, gamma)

(array([-10.,   5.]),
 array([-10.        ,  -0.33333333,  -6.        ,   5.        ]),
 np.False_)

In [97]:
get_g_star_q_star(1, 3, p_sp_given_s_a, r, gamma)

(array([3.18181818, 5.        ]),
 array([0.54545455, 3.18181818, 1.90909091, 5.        ]),
 np.True_)